In [1]:
import numpy as np
import networkx as nx
import time

In [2]:
# Import 2QAN compiler passes
from bench_arch import BenchArch
from heuristic_mapper import HeuristicMapper
from qrouter_depth import QuRouter

In [9]:
# Import qiskit 
from qiskit import transpile, QuantumCircuit

In [4]:
# run qaoa and run other trotter

## Run 2QAN compiler passes

In [10]:
def qs_compiler(qasm, coupling_map, lattice_xy, trials=1, mapper='qap', bgate='cx', basis_gates=None, recompile=True):
    qs_circ = None
    qs_swap = [0, 0]
    qs_g2 = 0
    if recompile:
        benchmark = QuantumCircuit.from_qasm_str(qasm)
        for trial in range(trials):
            # Initial qubit mapping 
            start = time.time()
            hmapper = HeuristicMapper(qasm, lattice_xy=lattice_xy, coupling_map=coupling_map)
            if mapper == 'qap':
                # The default mapper based on Quadratic Assignment Problem
                init_map, cost = hmapper.run_qap(num_iter=200, lst_len=20)
            elif mapper == 'qiskit':
                # The mapper in Qiskit
                init_map = hmapper.run_qiskit(max_iterations=5)
            end = time.time()
            print("Mapper run time: ", end - start)
            # Routing and scheduling, takes init_map as input
            # init_map = {circuit qubit index:device qubit index}
            print('The initial qubit map is \n', init_map)
            router = QuRouter(qasm, lattice_xy=lattice_xy, init_map=init_map, coupling_map=coupling_map)
            qs_circ0, swaps1 = router.run(layers=1, msmt='True')
            
            # Gate count 
            qs_circ1 = transpile(qs_circ0, basis_gates=None, optimization_level=3)
            g2_count1 = 0
            g2_count1 += qs_circ1.count_ops()[bgate]
            if 'unitary' in qs_circ1.count_ops():
                g2_count1 += qs_circ1.count_ops()['unitary']
            if 'swap' in qs_circ1.count_ops():
                g2_count1 += qs_circ1.count_ops()['swap']
            if trial == 0:
                qs_circ = qs_circ1
                qs_swap = swaps1
                qs_g2 = g2_count1 
            elif g2_count1 < qs_g2:
                qs_circ = qs_circ1
                qs_swap = swaps1
                qs_g2 = g2_count1
            print(g2_count1, qs_swap)
    else:
        qs_circ = transpile(qasm, basis_gates=basis_gates, optimization_level=3)
        qs_g2 += qs_circ.count_ops()[bgate] 
        if 'unitary' in qs_circ.count_ops().keys():
            qs_g2 += qs_circ.count_ops()['unitary']
    return qs_circ, qs_swap, qs_g2

## Tests

In [6]:
import os
import pickle as pkl
import qiskit
# Benchmarks
with open(os.path.join('qaoa_qasms.pkl'), 'rb') as f:
    qasms = pkl.load(f)
# with open(os.path.join('nnn_ising_qasms.pkl'), 'rb') as f:
#     qasms = pkl.load(f)
# qasms = []
# for c in qaoas:
#     qasms.append(c.qasm())
circs = [qiskit.QuantumCircuit.from_qasm_str(qasm) for qasm in qasms]
test_circ = circs[-2]
c_qasm = qasms[-2]
# with open(os.path.join('nnn_ising_qasms.pkl'), 'wb') as f:
#     pkl.dump(qasms, f)  

In [7]:
# Device information
# gate set
basis_gates = ['id', 'rz', 'u3', 'u2', 'cx', 'reset']

# topology
qn = len(test_circ.qubits)
dx = int(np.sqrt(qn))
print('The number of qubits is ', qn)
if dx*dx >= qn:
    lattice_xy = (dx, dx)
elif dx*(dx+1) >= qn:
    lattice_xy = (dx, dx+1)
elif dx*(dx+2) >= qn:
    lattice_xy = (dx, dx+2)
grid_topology = BenchArch(c_qasm, lattice_xy=lattice_xy).topology
coupling_map = [list(edge) for edge in list(grid_topology.edges)]
coupling_map += [[edge[1], edge[0]] for edge in list(grid_topology.edges)]

The number of qubits is  20


## QAP mapper + 2QAN routing&scheduling

In [11]:
qs_circ, qs_swap, qs_g2 = qs_compiler(c_qasm, coupling_map, lattice_xy, trials=5, bgate='rzz', basis_gates=basis_gates, recompile=True)
print('The number of SWAPs: ', qs_swap, qs_g2)

Mapper run time:  13.580049753189087
The inital qubit map is 
 {0: 2, 1: 13, 2: 12, 3: 14, 4: 5, 5: 18, 6: 4, 7: 6, 8: 11, 9: 10, 10: 8, 11: 9, 12: 7, 13: 3, 14: 17, 15: 19, 16: 0, 17: 15, 18: 1, 19: 16}
33 (8, 5)
Mapper run time:  13.711404800415039
The inital qubit map is 
 {0: 0, 1: 14, 2: 10, 3: 18, 4: 3, 5: 19, 6: 7, 7: 9, 8: 12, 9: 5, 10: 2, 11: 13, 12: 8, 13: 4, 14: 15, 15: 17, 16: 6, 17: 16, 18: 1, 19: 11}
32 (9, 7)
Mapper run time:  13.605650424957275
The inital qubit map is 
 {0: 9, 1: 5, 2: 12, 3: 1, 4: 17, 5: 0, 6: 19, 7: 15, 8: 7, 9: 14, 10: 16, 11: 6, 12: 11, 13: 10, 14: 4, 15: 2, 16: 18, 17: 3, 18: 13, 19: 8}
33 (9, 7)
Mapper run time:  13.490565776824951
The inital qubit map is 
 {0: 17, 1: 10, 2: 3, 3: 14, 4: 6, 5: 11, 6: 0, 7: 4, 8: 13, 9: 8, 10: 2, 11: 9, 12: 12, 13: 16, 14: 15, 15: 19, 16: 1, 17: 18, 18: 5, 19: 7}
35 (9, 7)
Mapper run time:  13.866186380386353
The inital qubit map is 
 {0: 10, 1: 13, 2: 16, 3: 14, 4: 4, 5: 18, 6: 0, 7: 1, 8: 7, 9: 2, 10: 12, 11: 5, 

In [12]:
qs_circ2, qs_swap, qs_g2 = qs_compiler(qs_circ, coupling_map, lattice_xy, bgate='cx', basis_gates=basis_gates, recompile=False)
print('The number of CNOTs: ', qs_g2)

The number of CNOTs:  73


## Qiskit SABRE mapper (by Gushu Li) + 2QAN routing&scheduling

In [15]:
ibm_circ, ibm_swap, ibm_g2 = qs_compiler(c_qasm, coupling_map, lattice_xy, trials=5, mapper='qiskit', bgate='rzz', basis_gates=basis_gates, recompile=True)
print('The number of SWAPs: ', ibm_swap, ibm_g2)

Mapper run time:  0.24879240989685059
The inital qubit map is 
 {0: 19, 1: 6, 2: 10, 3: 1, 4: 16, 5: 3, 6: 17, 7: 4, 8: 9, 9: 8, 10: 18, 11: 0, 12: 5, 13: 12, 14: 7, 15: 2, 16: 14, 17: 15, 18: 13, 19: 11}
38 (15, 7)
Mapper run time:  0.24345064163208008
The inital qubit map is 
 {0: 4, 1: 14, 2: 18, 3: 9, 4: 15, 5: 17, 6: 11, 7: 3, 8: 1, 9: 6, 10: 19, 11: 5, 12: 2, 13: 7, 14: 12, 15: 8, 16: 10, 17: 0, 18: 13, 19: 16}
37 (10, 3)
Mapper run time:  0.24016070365905762
The inital qubit map is 
 {0: 10, 1: 11, 2: 7, 3: 18, 4: 1, 5: 19, 6: 0, 7: 8, 8: 12, 9: 13, 10: 2, 11: 17, 12: 16, 13: 6, 14: 15, 15: 14, 16: 4, 17: 9, 18: 5, 19: 3}
34 (12, 8)
Mapper run time:  0.25244784355163574
The inital qubit map is 
 {0: 15, 1: 1, 2: 4, 3: 2, 4: 14, 5: 3, 6: 12, 7: 8, 8: 5, 9: 9, 10: 16, 11: 0, 12: 17, 13: 13, 14: 7, 15: 6, 16: 18, 17: 11, 18: 19, 19: 10}
39 (12, 8)
Mapper run time:  0.3171579837799072
The inital qubit map is 
 {0: 6, 1: 18, 2: 19, 3: 8, 4: 3, 5: 14, 6: 7, 7: 2, 8: 0, 9: 9, 10: 15, 1

In [16]:
ibm_circ2, ibm_swap, ibm_g2 = qs_compiler(ibm_circ, coupling_map, lattice_xy, bgate='cx', basis_gates=basis_gates, recompile=False)
print('The number of CNOTs: ', ibm_g2)

The number of CNOTs:  80


## Qiskit compiler

In [26]:
qiskit_circ = transpile(test_circ, basis_gates=basis_gates, coupling_map=coupling_map, optimization_level=3)
print('The number of CNOTs: ', qiskit_circ.count_ops()['cx'])

The number of CNOTs:  107
